# Generate Dataset from Your Documentation

![image](./imgs/‎GenAIEnterprises.‎015.png)

![image](./imgs/‎GenAIEnterprises.‎016.png)

![image](./imgs/‎GenAIEnterprises.‎017.png)

# Initial Setup

In [9]:
import boto3
import os
import openai
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from IPython.display import display, Markdown
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

def get_api_key(ssm_client, parameter_path):
    '''Get the OpenAI API key from the SSM Parameter Store'''
    try:
        response = ssm_client.get_parameter(
            Name=parameter_path,
            WithDecryption=True
        )
        return response['Parameter']['Value']
    except ssm_client.exceptions.ParameterNotFound:
        raise Exception(f'Parameter {parameter_path} not found in SSM Parameter Store')

# Create an SSM client using Boto3
region_name = os.getenv('AWS_REGION', 'us-east-1') 
ssm = boto3.client('ssm', region_name=region_name)

openai_api_key = get_api_key(ssm_client=ssm, parameter_path='/openai/api_key')
langchain_api_key = get_api_key(ssm_client=ssm, parameter_path='/langchain/api_key')


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
openai.api_key = openai_api_key

# Set the model variable based on the current date
llm_model = "gpt-3.5-turbo-16k"

# Create the vector store and embedding function
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory='docs/chroma/',
    embedding_function=embedding
)

# Model Evaluation

In [11]:
baseline_llm_model = "gpt-3.5-turbo"
ft_llm_model = "ft:gpt-3.5-turbo-0613:neurons-lab::7xi7PZeg"

In [12]:
import json

def load_data(path: str):
    fp = open(path, "r")
    data_dicts = []
    for line in fp:
        d = json.loads(line)
        data_dicts.append(d)
    return data_dicts

train_dicts = load_data("data/qa_pairs_train.jsonl")
eval_dicts = load_data("data/qa_pairs_val.jsonl")

## Manual Evaluation

In [82]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={'fetch_k': 5, 'k': 7})

def query_model(model, query, retriever=None, temperature=0):
    """Query the model with the given data and return the result.
    
    Args:
        model (str): The model name to use.
        query: The query to use.
        retriever (Retriever): The retriever to use.
        temperature (float): The temperature to use when querying the model.
    
    Returns:
        dict: The query and result."""

    llm = ChatOpenAI(model_name=model, temperature=temperature)

    if not retriever:
        messages = [
            SystemMessage(
                content="Provide answers to questions based on the company handbook to help employees quickly find the information they need. Ensure that your responses are concise and directly address the questions asked without providing additional information."
            ),
            HumanMessage(
                content=query
            ),
        ]
        result = llm(messages)
        return result.content
    else:
        qa_chain = RetrievalQA.from_chain_type(
            llm,
            retriever=retriever,
            return_source_documents=True,
        )
        result = qa_chain(query)
        return result["result"]
        
        


In [132]:

query = train_dicts[48]['query']
expected_response = train_dicts[48]['response']

display(Markdown("## Query:"))
display(Markdown(query))
display(Markdown("## Expected Response:"))
display(Markdown(expected_response))

display(Markdown("## Baseline LLM:"))
baseline_result = query_model(baseline_llm_model, query)
display(Markdown(f"### Result: \n {baseline_result}"))

display(Markdown("## FineTuned LLM:"))
ft_result = query_model(ft_llm_model, query)
display(Markdown(f"### Result: \n {ft_result}"))

display(Markdown("## FineTuned LLM + RAG:"))
ft_rag_result = query_model(ft_llm_model, query, retriever=retriever)
display(Markdown(f"### Result: \n {ft_rag_result}"))



## Query:

What is the purpose of a buddy in the Made Tech company?

## Expected Response:

The purpose of a buddy in the Made Tech company is to help new team members connect and find their way, especially during their first 3 months with the company. Buddies are friendly and helpful individuals who proactively check in with new starters, offer their help and answer questions, explain how things work in the company, and provide guidance and support. They play an important role in helping new team members integrate into Made Tech.

## Baseline LLM:

### Result: 
 The purpose of a buddy in Made Tech is to provide support and guidance to new employees during their onboarding process.

## FineTuned LLM:

### Result: 
 The purpose of a buddy in the Made Tech company is to provide support and guidance to new joiners during their first few weeks. Buddies are responsible for helping new joiners settle in, answering any questions they may have, and assisting them in finding the right people to speak to. They also organize regular catch-ups with new joiners to check on their progress and well-being. Overall, buddies play a crucial role in making new joiners feel welcome and helping them navigate their initial experiences at Made Tech.

## FineTuned LLM + RAG:

### Result: 
 The purpose of a buddy in the Made Tech company is to help new team members connect and find their way, especially in their first 3 months with the company when everything is new to them. Buddies are friendly individuals who provide support and guidance to new team members, both in-person when back in the office (e.g., showing them good lunch places, taking them out for coffee or lunch, helping them navigate the local area) and remotely through video calls or "coffee chats". Being a buddy is an important role in helping new team members integrate into Made Tech.

## Embedding Distance

To measure semantic similarity (or dissimilarity) between a prediction and a reference label string, you could use a vector vector distance metric the two embedded representations using the embedding_distance evaluator.

Note: This returns a **distance** score, meaning that the lower the number, the **more** similar the prediction is to the reference, according to their embedded representation.

In [154]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.evaluation import load_evaluator

embedding_model = HuggingFaceEmbeddings()
hf_evaluator = load_evaluator("embedding_distance", embeddings=embedding_model)


In [155]:
hf_evaluator.evaluate_strings(prediction="I shall go", reference="I shan't go")

{'score': 0.5486443280477362}

In [156]:
hf_evaluator.evaluate_strings(prediction="I shall go", reference="I will go")

{'score': 0.21018880025138598}

In [142]:
display(Markdown("## Baseline LLM:"))
display(hf_evaluator.evaluate_strings(prediction=baseline_result, reference=expected_response))

display(Markdown("## FineTuned LLM:"))
display(hf_evaluator.evaluate_strings(prediction=ft_result, reference=expected_response))

display(Markdown("## FineTuned LLM + RAG:"))
display(hf_evaluator.evaluate_strings(prediction=ft_rag_result, reference=expected_response))

## Baseline LLM:

{'score': 0.060356865075396504}

## FineTuned LLM:

{'score': 0.025603949751298205}

## FineTuned LLM + RAG:

{'score': 0.02472507271870661}

In [90]:
import pandas as pd
from tqdm.notebook import tqdm

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

pd.set_option("display.max_colwidth", None)

EVAL_PROMPT_TMPL = PromptTemplate.from_template(
    """\
We provide a question and the 'ground-truth' answer. We also provide \
the predicted answer.

Evaluate whether the predicted answer is correct, given its similarity \
to the ground-truth. If details provided in predicted answer are reflected \
in the ground-truth answer, return "YES". To return "YES", the details don't \
need to exactly match. Be lenient in evaluation if the predicted answer \
is missing a few details. Try to make sure that there are no blatant mistakes. \
Otherwise, return "NO".

Question: {question}
Ground-truth Answer: {gt_answer}
Predicted Answer: {pred_answer}
Evaluation Result: \
"""
)


def eval_match_gt(query, gt_response, pred_response):
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    fmt_prompt = EVAL_PROMPT_TMPL.format(
        question=query,
        gt_answer=gt_response,
        pred_answer=pred_response,
    )

    messages = [
        HumanMessage(
            content=fmt_prompt
        ),
    ]

    result = llm(messages)
    if "yes" in str(result).lower():
        return 1
    else:
        return 0


def run_evals(eval_dicts):
    """Run evals - fine-tuned model, RAG system, and base model."""

    raw_responses = []
    for eval_dict in tqdm(eval_dicts):
        print(eval_dict)
        gt_response = eval_dict["response"]
        query_model(ft_llm_model, train_dicts[48], retriever=retriever)
        ft_rag_response = str(query_model(ft_llm_model, eval_dict["query"], retriever=retriever))
        ft_response = str(query_model(ft_llm_model, eval_dict["query"]))
        rag_response = str(query_model(baseline_llm_model, eval_dict["query"], retriever=retriever))
        base_response = str(query_model(baseline_llm_model, eval_dict["query"]))

        # try evaluations
        ft_rag_eval = eval_match_gt(eval_dict["query"], gt_response, ft_rag_response)
        ft_eval = eval_match_gt(eval_dict["query"], gt_response, ft_response)
        rag_eval = eval_match_gt(eval_dict["query"], gt_response, rag_response)
        base_eval = eval_match_gt(eval_dict["query"], gt_response, base_response)

        response_dict = {
            "query": eval_dict["query"],
            "gt_response": gt_response,
            "ft_rag_response": ft_rag_response,
            "ft_response": ft_response,
            "rag_response": rag_response,
            "base_response": base_response,
            "ft_rag_eval": ft_rag_eval,
            "ft_eval": ft_eval,
            "rag_eval": rag_eval,
            "base_eval": base_eval,
        }

        raw_responses.append(response_dict)

    raw_responses_df = pd.DataFrame(raw_responses)

    eval_dict = {
        "ft_rag_score": raw_responses_df["ft_rag_eval"].mean(),
        "ft_score": raw_responses_df["ft_eval"].mean(),
        "rag_score": raw_responses_df["rag_eval"].mean(),
        "base_score": raw_responses_df["base_eval"].mean(),
    }

    sub_responses_df = raw_responses_df[
        [
            "query",
            "gt_response",
            "ft_rag_response",
            "ft_response",
            "rag_response",
            "base_response",
        ]
    ]

    return eval_dict, raw_responses_df, sub_responses_df

In [94]:
eval_dict, raw_response_df, sub_responses_df = run_evals(eval_dicts)

  0%|          | 0/100 [00:00<?, ?it/s]

{'query': 'What is the policy at Made Tech regarding false allegations of discrimination?', 'response': 'Making a false allegation deliberately and in bad faith at Made Tech will be treated as misconduct.'}
{'query': 'What types of working hours patterns are supported by Made Tech?', 'response': 'Made Tech supports reduced working hours patterns, but does not support compressed hours or other patterns such as annualised hours.'}
{'query': 'What is the other path to a salary adjustment at Made Tech apart from the annual review?', 'response': 'The other path to a salary adjustment at Made Tech apart from the annual review is through the Internal mobility process, which includes promotions or lateral moves within the financial year.'}
{'query': 'What happens if the 28th of the month falls on a weekend or during a bank holiday at Made Tech?', 'response': 'If the 28th of the month falls on a weekend or during a bank holiday at Made Tech, the payday is moved to the Friday before.'}
{'query':

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'query': "How should chalet members record their time spent on 'New business and revenue' activities?", 'response': "Chalet members should record their time spent on 'New business and revenue' activities as 'Bench/Chalet' in their timesheets."}
{'query': 'What are the expectations for Made Tech team members while working remotely in terms of team ceremonies and Slack communication?', 'response': 'While working remotely, Made Tech team members are expected to ensure that they are available for team ceremonies as normal and are able to dial in without interruption. They are also expected to be a good Slack citizen and check in regularly and visibly with their team throughout the working day in public channels. Additionally, they should ensure that they are not expected for any in-person meetings or sessions while working remotely.'}
{'query': 'What consequences may occur if there is a violation of the equality, diversity, and inclusion policy at Made Tech?', 'response': 'Violations of t

Failed to patch https://api.langchain.plus/runs/307efa4c-0157-4851-9dae-7c7c8486bb85 in LangSmith API. 400 Client Error: Bad Request for url: https://api.langchain.plus/runs/307efa4c-0157-4851-9dae-7c7c8486bb85
{"detail":"Cannot update a run that has already finished"}


...

{'query': "What activities are encouraged during 'Chalet Time' at Made Tech?", 'response': "During 'Chalet Time' at Made Tech, activities that are encouraged include building skills, contributing to Communities of Practice, and growing Made Tech's business. It is a time to focus on other activities that are not directly related to client work."}
{'query': 'What are the approval processes in place for expense claims at Made Tech?', 'response': 'Claims that align with the policy will be approved, but any claims that are outside of the guidelines will need approval from the Head of Department. Expenses that are aligned to a specific delivery or account should be agreed with the Delivery Manager and/or account team in advance. Expenses for events and conferences need to be approved in advance by the relevant Head of Department.'}
{'query': 'What is the preferred format for internal office documents at Made Tech?', 'response': 'The preferred format for internal office documents at Made Tech

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'query': 'How can chalet members contribute to new revenue and business as per the Made Tech handbook?', 'response': 'Chalet members can contribute to new revenue and business by participating in various activities outlined in the Made Tech handbook. The Bids team will provide details of bids that chalet members can contribute to, allowing them to contribute to the revenue generation process. Additionally, the Marketing team will share tasks that chalet members can do to promote Made Tech, such as creating blog posts or contacting relevant teams for input. These tasks will be posted in the Chalet Community Board, and chalet members can engage in them to contribute to the growth of the business.'}
{'query': 'What platforms should every member of Made Tech have access to, with two-factor authentication?', 'response': 'Every member of Made Tech should have access to Google Mail, Github, Slack, 1Password, Trello (unless using Google Auth to login to Trello), and HiBob, all with two-factor

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'query': 'What is the process to submit expense claims at Made Tech?', 'response': "Expense claims at Made Tech need to be submitted via Xero. If an employee does not have a login for Xero, they can drop a message in #ops-finance or email finance@madetech.com to request one. All expense claims must be made within 90 days and will typically be paid within 2 weeks. There are different approval processes in place depending on the type of expense. Normal expenses that align with the policy will be approved, but any claims that are outside of the guidelines will require approval from the employee's Head of Department. Expenses that are aligned to a specific delivery or account should be agreed upon with the Delivery Manager and/or account team in advance. Expenses for events and conferences need to be approved in advance by the relevant Head of Department. For any other information, employees should refer to the policy document and direct any questions to #ops-finance or finance@madetech.c

In [95]:
display(eval_dict)
display(sub_responses_df)

{'ft_rag_score': 0.84, 'ft_score': 0.59, 'rag_score': 0.75, 'base_score': 0.46}

,query,gt_response,ft_rag_response,ft_response,rag_response,base_response
0,What is the policy at Made Tech regarding false allegations of discrimination?,Making a false allegation deliberately and in bad faith at Made Tech will be treated as misconduct.,"Making a false allegation deliberately and in bad faith will be treated as misconduct according to Made Tech's equality, diversity, and inclusion policy.",Made Tech takes false allegations of discrimination seriously and they may result in disciplinary action.,Making a false allegation deliberately and in bad faith will be treated as misconduct according to Made Tech's policy.,"Made Tech takes false allegations of discrimination very seriously. Any employee found to have made a false allegation of discrimination may be subject to disciplinary action, up to and including termination of employment."
1,What types of working hours patterns are supported by Made Tech?,"Made Tech supports reduced working hours patterns, but does not support compressed hours or other patterns such as annualised hours.","Made Tech supports a variety of working hours patterns, including but not limited to:\n- 9-day fortnight\n- Part-time\n- Job share\n- Flexible hours\n- Remote working\n\nThey also offer a flexible benefits package that allows individuals to choose the benefits that suit their lifestyle, including the option to buy and sell up to 5 days of annual leave.","Made Tech supports a variety of working hours patterns, including full-time, part-time, and job share arrangements. They also offer flexible working hours, allowing employees to start and finish work at different times.",The provided context does not mention specific working hour patterns supported by Made Tech. It is best to reach out to Made Tech directly for information on their supported working hour patterns.,"Made Tech supports flexible working hours patterns, including full-time, part-time, and job-sharing arrangements."
2,What is the other path to a salary adjustment at Made Tech apart from the annual review?,"The other path to a salary adjustment at Made Tech apart from the annual review is through the Internal mobility process, which includes promotions or lateral moves within the financial year.","The other path to a salary adjustment at Made Tech apart from the annual review is through the Internal mobility process, which includes promotions or lateral moves within the financial year.",The other path to a salary adjustment at Made Tech apart from the annual review is through the promotion process.,"The other path to a salary adjustment at Made Tech is through the Internal mobility process, which includes promotions or lateral moves within the financial year.","Employees can request a salary adjustment outside of the annual review process by submitting a formal request to their line manager. This request should include a clear justification for the adjustment, such as significant changes in job responsibilities or market conditions. The line manager will review the request and make a decision based on the company's salary adjustment policy."
3,What happens if the 28th of the month falls on a weekend or during a bank holiday at Made Tech?,"If the 28th of the month falls on a weekend or during a bank holiday at Made Tech, the payday is moved to the Friday before.","If the 28th of the month falls on a weekend or during a bank holiday at Made Tech, the payday will be moved to the last working day before the 28th.","If the 28th of the month falls on a weekend or during a bank holiday at Made Tech, the company will pay employees on the last working day before the 28th.","If the 28th of the month falls on a weekend or during a bank holiday at Made Tech, the holiday requests will be reviewed and confirmed on the next working day after the weekend or bank holiday.","If the 28th of the month falls on a weekend or during a bank holiday at Made Tech, the salary payment will be made on the last working day before the 28th."
4,What happ

# Evaluate LLM models in LangSmith

## Create LangSmith LLM Dataset

In [96]:

from langsmith import Client

client = Client()
dataset = client.create_dataset(dataset_name="MadeTech Dataset", description="Made Tech Handbook dataset with LLM inputs and outputs", data_type="llm")

for eval_dict in eval_dicts:
    client.create_example(
        inputs={"input": eval_dict["query"]},
        outputs={"output": eval_dict["response"]},
        dataset_id=dataset.id,
        # Or dataset_name="My LLM Dataset"
    )

## Evaluate BaseLine LLM in LangSmith

In [ ]:
import langsmith
from langchain.chat_models import ChatOpenAI
from langchain import smith

# Replace with the LLM you want to test
my_llm = ChatOpenAI(model=baseline_llm_model,temperature=0)

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=[
        "cot_qa"
    ],
    custom_evaluators=[],
    eval_llm=ChatOpenAI(model="gpt-4", temperature=0)
)

client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="MadeTech Dataset",
    llm_or_chain_factory=my_llm,
    evaluation=eval_config,
    project_name="test-baseline-llm-22",
    verbose=True,
    tags=["baseline", "llm"],
)

## Evaluate Fine Tuned LLM in LangSmith

In [109]:
import langsmith
from langchain.chat_models import ChatOpenAI
from langchain import smith

# Replace with the LLM you want to test
my_llm = ChatOpenAI(model=ft_llm_model, temperature=0)

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=[
        "cot_qa"
    ],
    custom_evaluators=[],
    eval_llm=ChatOpenAI(model="gpt-4", temperature=0)
)

client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="MadeTech Dataset",
    llm_or_chain_factory=my_llm,
    evaluation=eval_config,
    project_name="test-fine-tuned-llm-23",
    verbose=True,
    tags=["ft", "llm"],
)

View the evaluation results for project 'test-fine-tuned-llm-23' at:
https://smith.langchain.com/o/f09d95d4-78e8-5dcb-a136-09de2b589be4/projects/p/57515a66-a598-465c-9871-6e9bdd9d3790
[--------------------------------------->          ] 81/100

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-f5H7bPv9fgPptJoKt2f4cPHG on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..


[------------------------------------------->      ] 89/100

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-f5H7bPv9fgPptJoKt2f4cPHG on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..


[--------------------------------------------->    ] 93/100

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-f5H7bPv9fgPptJoKt2f4cPHG on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..


[------------------------------------------------->] 100/100
 Eval quantiles:
                         0.25  0.50  0.75
COT Contextual Accuracy   0.0   0.0   1.0


# RAG Evaluation

## Create LangSmith KV Dataset

In [110]:
from langsmith import Client

client = Client()
dataset = client.create_dataset(dataset_name="MadeTech Dataset KV", description="Made Tech Handbook dataset with LLM inputs and outputs", data_type="kv")

for eval_dict in eval_dicts:
    client.create_example(
        inputs={"input": eval_dict["query"]},
        outputs={"output": eval_dict["response"]},
        dataset_id=dataset.id,
        # Or dataset_name="My LLM Dataset"
    )

## Evaluate BaseLine RAG in LangSmith

In [116]:
import langsmith
from langchain.chat_models import ChatOpenAI
from langchain import smith

retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={'fetch_k': 5, 'k': 7})
llm = ChatOpenAI(model_name=baseline_llm_model, temperature=0)

def chain_constructor():
    return RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
)

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=[
        "cot_qa"
    ],
    custom_evaluators=[],
    eval_llm=ChatOpenAI(model="gpt-4", temperature=0)
)

client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="MadeTech Dataset KV",
    llm_or_chain_factory=chain_constructor,
    evaluation=eval_config,
    project_name="test-rag-baseline-27",
    verbose=True,
    tags=["rag", "baseline"],
)


View the evaluation results for project 'test-rag-baseline-27' at:
https://smith.langchain.com/o/f09d95d4-78e8-5dcb-a136-09de2b589be4/projects/p/8af439de-859f-47df-8ec3-c092afa78729
[------------------------------------------------> ] 98/100

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


[------------------------------------------------->] 99/100

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


[------------------------------------------------->] 100/100
 Eval quantiles:
                         0.25  0.50  0.75
COT Contextual Accuracy   0.0   1.0   1.0


## Evaluate Fine Tune RAG in LangSmith

In [121]:
import langsmith
from langchain.chat_models import ChatOpenAI
from langchain import smith

retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={'fetch_k': 5, 'k': 7})
llm = ChatOpenAI(model_name=ft_llm_model, temperature=0)

def chain_constructor():
    return RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
)

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=[
        "cot_qa"
    ],
    custom_evaluators=[],
    eval_llm=ChatOpenAI(model="gpt-4", temperature=0)
)

client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="MadeTech Dataset KV",
    llm_or_chain_factory=chain_constructor,
    evaluation=eval_config,
    project_name="test-rag-ft-28",
    verbose=True,
    tags=["ft", "rag"],
)

View the evaluation results for project 'test-rag-ft-28' at:
https://smith.langchain.com/o/f09d95d4-78e8-5dcb-a136-09de2b589be4/projects/p/c9be9a2b-0c22-4e97-9512-e605d9b6a17d
[----------------------->                          ] 48/100

Failed to post https://api.langchain.plus/runs in LangSmith API. 409 Client Error: Conflict for url: https://api.langchain.plus/runs
{"detail":"Run with this id already exists"}
Error in LangChainTracer.on_chat_model_start callback: Failed to post https://api.langchain.plus/runs in LangSmith API. 409 Client Error: Conflict for url: https://api.langchain.plus/runs
{"detail":"Run with this id already exists"}
Error in LangChainTracer.on_chat_model_start callback: Failed to post https://api.langchain.plus/runs in LangSmith API. 409 Client Error: Conflict for url: https://api.langchain.plus/runs
{"detail":"Run with this id already exists"}
Error in LangChainTracer.on_chain_end callback: Failed to patch https://api.langchain.plus/runs/11013266-88cc-4d08-a861-80dfa6beb8cf in LangSmith API. 400 Client Error: Bad Request for url: https://api.langchain.plus/runs/11013266-88cc-4d08-a861-80dfa6beb8cf
{"detail":"Cannot update a run that has already finished"}


[----------------------->                          ] 49/100

Error in LangChainTracer.on_chain_start callback: Failed to post https://api.langchain.plus/runs in LangSmith API. 409 Client Error: Conflict for url: https://api.langchain.plus/runs
{"detail":"Run with this id already exists"}


[------------------------------------------->      ] 87/100

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


[------------------------------------------------->] 100/100
 Eval quantiles:
                         0.25  0.50  0.75
COT Contextual Accuracy   0.0   1.0   1.0
